In [17]:
import os
import pickle
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

In [18]:
path = '/data/notebook/shared/MIMIC-IV'

In [19]:
with open(os.path.join(path, 'total_data_dict_with_timedelta_nomedi_with_code_label_240423_clinic_3_years.pkl'), 'rb') as f:
    data_dict_d = pickle.load(f)
f.close()

In [20]:
total_labels = []
code_labels = []
clinical_labels = []
# code_seq = []
# time_delta_list = []

for sample_id, visits in tqdm(data_dict_d.items()):
    # 레이블 추가
    total_label = visits['total_label']
    code_label = visits['code_label']
    clinical_label = visits['clinical_label']
    total_labels.append(total_label)
    code_labels.append(code_label)
    # clinical_labels.append(clinical_labels)
    # code_seq.append([visits['year']])
    # time_delta_list.append(visits['timedelta'])

100%|██████████| 8037/8037 [00:00<00:00, 792421.75it/s]


In [24]:
train_indices, test_indices, train_y, test_y = train_test_split(list(data_dict_d.keys()), total_labels, test_size=0.1, random_state=777, stratify=total_labels)
train_indices, valid_indices, valid_y, valid_y = train_test_split(train_indices, train_y, test_size=(len(test_indices)/len(train_indices)), random_state=777, stratify=train_y) 

In [25]:
train_data = {}
valid_data = {}
test_data = {}
for sample in tqdm(train_indices):
    train_data[sample] = data_dict_d[sample]

for sample in tqdm(valid_indices):
    valid_data[sample] = data_dict_d[sample]

for sample in tqdm(test_indices):
    test_data[sample] = data_dict_d[sample]

100%|██████████| 804/804 [00:00<00:00, 865559.66it/s]


In [45]:
tr_labels = []
tr_code_labels = []
tr_clinical_labels = []
tr_code_seq = []
tr_time_delta_list = []

for sample_id, visits in tqdm(train_data.items()):
    # 레이블 추가
    total_label = visits['total_label']
    code_label = visits['code_label']
    clinical_label = visits['clinical_label']
    tr_labels.append(total_label)
    tr_code_labels.append(code_label)
    tr_clinical_labels.append(clinical_labels)
    tr_code_seq.extend([visits['year']])
    tr_time_delta_list.append(visits['timedelta'])

100%|██████████| 6429/6429 [00:00<00:00, 440370.07it/s]


In [46]:
val_labels = []
val_code_labels = []
val_clinical_labels = []
val_code_seq = []
val_time_delta_list = []

for sample_id, visits in tqdm(valid_data.items()):
    # 레이블 추가
    total_label = visits['total_label']
    code_label = visits['code_label']
    clinical_label = visits['clinical_label']
    val_labels.append(total_label)
    val_code_labels.append(code_label)
    val_clinical_labels.append(clinical_labels)
    val_code_seq.extend([visits['year']])
    val_time_delta_list.append(visits['timedelta'])

100%|██████████| 804/804 [00:00<00:00, 394734.92it/s]


In [47]:
te_labels = []
te_code_labels = []
te_clinical_labels = []
te_code_seq = []
te_time_delta_list = []

for sample_id, visits in tqdm(test_data.items()):
    # 레이블 추가
    total_label = visits['total_label']
    code_label = visits['code_label']
    clinical_label = visits['clinical_label']
    te_labels.append(total_label)
    te_code_labels.append(code_label)
    te_clinical_labels.append(clinical_labels)
    te_code_seq.extend([visits['year']])
    te_time_delta_list.append(visits['timedelta'])

100%|██████████| 804/804 [00:00<00:00, 382815.35it/s]


In [48]:
hitanet_train = [tr_code_seq, tr_labels, tr_time_delta_list]
hitanet_valid = [val_code_seq, val_labels, val_time_delta_list]
hitanet_test = [te_code_seq, te_labels, te_time_delta_list]

In [49]:
with open('./data/Hitanet/train_HT_777.pkl', 'wb') as f:
    pickle.dump(hitanet_train, f)

with open('./data/Hitanet/valid_HT_777.pkl', 'wb') as f:
    pickle.dump(hitanet_valid, f)
    
with open('./data/Hitanet/test_HT_777.pkl', 'wb') as f:
    pickle.dump(hitanet_test, f)

In [44]:
hitanet_train[0][0][0]

[[638, 6707, 8107, 812, 10506, 812, 10506, 812, 10506],
 [5627, 6937, 5425, 12701, 12701, 12701],
 [2323, 1622, 5958, 12701, 12701, 12701]]

In [40]:
len(hitanet_train[2])

6429

In [7]:
np.unique(total_label, return_counts=True)
np.mean(length_list)
np.std(length_list)

# max_visits_length = max(length_list)
max_index = max(dtype_dict.values())
max_code_len = max(code_length_list)
# print('max_index:', max_index+1)
print('max_visit:', max_visits_length)
print('max_code_len:', max_code_len)

max_visit: 49
max_code_len: 27


In [121]:
def pad_sequence(seq_diagnosis_codes, maxlen, maxcode):
    lengths = len(data['year'])
    diagnosis_codes = np.zeros((maxlen, maxcode), dtype=np.int64)
    seq_mask_code = np.zeros((maxlen, maxcode), dtype=np.int8)
    seq_mask = np.zeros((maxlen), dtype=np.int8)
    seq_mask_final = np.zeros((maxlen), dtype=np.int8)
    for pid, subseq in enumerate(seq_diagnosis_codes):
        for tid, code in enumerate(subseq):
            diagnosis_codes[pid, tid] = code
            seq_mask_code[pid, tid] = 1
    seq_mask[:lengths] = 1
    seq_mask_final[lengths - 1] = 1
    return diagnosis_codes, seq_mask_code, seq_mask, seq_mask_final

In [122]:
def keep_last_one_in_columns(a):
    # 결과 배열 초기화
    result = np.zeros_like(a)
    # 각 열에 대해 반복
    for col_index in range(a.shape[1]):
        # 현재 열 추출
        column = a[:, col_index]
        # 이 열에서 마지막 '1' 찾기
        last_one_idx = np.max(np.where(column == 1)[0]) if 1 in column else None
        if last_one_idx is not None:
            result[last_one_idx, col_index] = 1
    return result

In [123]:
new_data_dict_d = {}
year_list = []
for sample_id, data in tqdm(data_dict_d.items()):
    data_dict_new = {}
    # pad_seq, seq_mask_code = pad_sequence(data['year'], max_visits_length, max_code_len)
    pad_seq, seq_mask_code, seq_mask, seq_mask_final = pad_sequence(data['year'], max_visits_length, max_code_len)
    data_dict_new['code_index'] = pad_seq
    data_dict_new['code'] = data['code']
    data_dict_new['time'] = data['time']
    data_dict_new['timedelta'] = data['timedelta']
    time_feature = np.array([[timestamp.year, timestamp.month, timestamp.day, timestamp.week] for timestamp in data['time']])
    data_dict_new['time_feature'] = np.pad(time_feature, pad_width=((0, max_visits_length - time_feature.shape[0]),(0,0)))
    data_dict_new['year'] = np.array([timestamp.year for timestamp in data['time']])
    # data_dict_new['seq_mask'] = np.pad(np.ones(time_feature.shape[0]), (0, max_visits_length - time_feature.shape[0]))
    data_dict_new['seq_mask'] = seq_mask
    data_dict_new['seq_mask_final'] = seq_mask_final
    data_dict_new['seq_mask_code'] = seq_mask_code
    unique_year = np.unique(data_dict_new['year'])
    if len(unique_year) == 2:
        # print("two year sample", sample_id)
        unique_year = np.append(unique_year, unique_year[-1]+1)
    elif len(unique_year) == 1:
        # print("one year sample", sample_id)
        continue
    encoder = OneHotEncoder(categories=[unique_year], sparse=False, handle_unknown='ignore')
    year_onehot =  encoder.fit_transform(np.array(data_dict_new['year']).reshape(-1,1))
    last_year_visit = keep_last_one_in_columns(year_onehot)
    data_dict_new['year_onehot'] = np.pad(year_onehot, pad_width=((0, max_visits_length - year_onehot.shape[0]), (0,0)))
    data_dict_new['last_year_onehot'] = np.pad(last_year_visit, pad_width=((0, max_visits_length - year_onehot.shape[0]), (0,0)))
    data_dict_new['label'] = data['total_label']    
    data_dict_new['code_label'] = data['code_label']    
    data_dict_new['clinical_label'] = data['clinical_label']    
    new_data_dict_d[sample_id] = data_dict_new

100%|██████████| 8037/8037 [00:02<00:00, 2828.19it/s]


In [124]:
with open(os.path.join('./data/', 'preprocessed_nomedi_240423_clinic_3_years.pkl'), 'wb') as f:
    pickle.dump(new_data_dict_d, f)
f.close()

In [107]:
os.path.join(path, 'total_data_dict_with_timedelta_nomedi_240421_clinic_3_years.pkl')

'/data/notebook/shared/MIMIC-IV/total_data_dict_with_timedelta_nomedi_240421_clinic_3_years.pkl'

In [108]:
np.outer(seq_mask, seq_mask)

array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int8)

In [ ]:
[[1, 0, 0, 0, 0],
 [1, 1, 0, 0, 0],
 [0, 1, 1, 0, 0],
 [0, 0, 1, 1, 0],
 [0, 0, 0, 1, 1]]